In [1]:
import pandas as pd
import numpy as np
import data_preprocessing as dp
import mdp
import Replay_Buffer
import torch as T

### load training data

In [2]:
# load data: DR_USA_Roundabout_FT_train.csv

# Roundabout data google drive path: "https://drive.google.com/file/d/1Ls92QosBDIyoDcEudSVGukP68pw1FRXW/view?usp=sharing"
# please download it to your local computer

url_FT_train = 'INTERACTION/DR_USA_Roundabout_FT_train.csv'
df_train = pd.read_csv(url_FT_train)

# look at the top 30 rows of data
df_train.head(30)

,case_id,track_id,frame_id,timestamp_ms,agent_type,x,y,vx,vy,psi_rad,length,width
0,1.0,1,1,100,pedestrian/bicycle,1044.509,1012.175,0.103,-1.005,NaN,NaN,NaN
1,1.0,2,1,100,pedestrian/bicycle,1043.840,1013.348,-0.101,-1.362,NaN,NaN,NaN
2,1.0,3,1,100,pedestrian/bicycle,1007.638,1022.038,0.687,0.787,NaN,NaN,NaN
3,1.0,4,1,100,car,1036.888,976.415,-1.063,1.980,2.064,5.20,1.86
4,1.0,5,1,100,car,1038.206,993.477,-2.343,5.161,1.997,4.68,1.90
5,1.0,6,1,100,car,1004.449,984.315,1.823,-0.369,-0.200,4.93,1.97
6,1.0,7,1,100,car,1013.407,987.769,4.564,-3.386,-0.638,4.83,1.93
7,1.0,8,1,100,car,1048.917,980.407,-0.894,0.591,2.557,5.13,1.96
8,1.0,9,1,100,car,993.728,985.780,1.184,0.012,0.010,4.54,1.81
9,1.0,10,1,100,car,981.026,985.788,4.411,0.197,0.045,4.55,1.78


### 1. create unique_id
### 2. change length, width and psi_rad from NaN to 0 for pedestrain/bicycle
### 3. change agent type to number for state vector (pedestrian/bicycle = -1; car = 1)

In [3]:
# add a new label unique_id using case_id and track_id
df_train['unique_id'] = df_train['case_id'].astype(int) * 100+ df_train['track_id']

# change length, width and psi_rad from NaN to 0 for pedestrain/bicycle
df_train.loc[df_train['agent_type'] == 'pedestrian/bicycle', 'length'] = 0
df_train.loc[df_train['agent_type'] == 'pedestrian/bicycle', 'width'] = 0
df_train.loc[df_train['agent_type'] == 'pedestrian/bicycle', 'psi_rad'] = 0


### important!!!
# agent_type pedestrain/bicycle = -1
# agent_type car = 1
df_train.loc[df_train['agent_type'] == 'pedestrian/bicycle', 'agent_type'] = -1
df_train.loc[df_train['agent_type'] == 'car', 'agent_type'] = 1
df_train

,case_id,track_id,frame_id,timestamp_ms,agent_type,x,y,vx,vy,psi_rad,length,width,unique_id
0,1.0,1,1,100,-1,1044.509,1012.175,0.103,-1.005,0.000,0.00,0.00,101
1,1.0,2,1,100,-1,1043.840,1013.348,-0.101,-1.362,0.000,0.00,0.00,102
2,1.0,3,1,100,-1,1007.638,1022.038,0.687,0.787,0.000,0.00,0.00,103
3,1.0,4,1,100,1,1036.888,976.415,-1.063,1.980,2.064,5.20,1.86,104
4,1.0,5,1,100,1,1038.206,993.477,-2.343,5.161,1.997,4.68,1.90,105
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4243282,9811.0,8,40,4000,1,1022.107,985.137,7.470,1.627,0.215,4.19,1.87,981108
4243283,9811.0,9,40,4000,1,996.468,986.201,7.374,-0.178,-0.024,4.64,1.90,981109
4243284,9811.0,10,40,4000,1,1019.894,1009.985,-7.780,-0.610,-3.063,4.72,1.87,981110
4243285,9811.0,11,40,4000,1,1028.592,1022.465,-1.619,-1.729,-2.324,4.49,1.83,981111


### use 10 cases (case1 to case10) for demonstrating and testing

In [4]:
# use case 1 to case 100 as a toy test dataframe
number_of_cases = 9800

df_train_small = df_train[df_train['case_id'] <= number_of_cases]
df_train_small 

,case_id,track_id,frame_id,timestamp_ms,agent_type,x,y,vx,vy,psi_rad,length,width,unique_id
0,1.0,1,1,100,-1,1044.509,1012.175,0.103,-1.005,0.000,0.00,0.00,101
1,1.0,2,1,100,-1,1043.840,1013.348,-0.101,-1.362,0.000,0.00,0.00,102
2,1.0,3,1,100,-1,1007.638,1022.038,0.687,0.787,0.000,0.00,0.00,103
3,1.0,4,1,100,1,1036.888,976.415,-1.063,1.980,2.064,5.20,1.86,104
4,1.0,5,1,100,1,1038.206,993.477,-2.343,5.161,1.997,4.68,1.90,105
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4238389,9800.0,14,40,4000,1,987.754,985.433,5.449,0.071,0.013,5.24,2.00,980014
4238390,9800.0,15,40,4000,1,1055.598,976.196,-1.233,0.867,2.529,4.33,1.81,980015
4238391,9800.0,16,40,4000,1,1040.531,969.451,0.000,0.000,2.100,5.02,2.06,980016
4238392,9800.0,17,40,4000,1,973.104,985.236,4.229,0.202,0.048,4.54,1.89,980017


In [5]:
print('--df--')
print('Total number of cases: ', df_train['case_id'].nunique())
print('Total number of agents: ', df_train.unique_id.nunique())
print('--df_small--')
print('Total number of cases: ', df_train_small['case_id'].nunique())
print('Total number of agents: ', df_train_small.unique_id.nunique())

--df--
Total number of cases:  9811
Total number of agents:  129019
--df_small--
Total number of cases:  9800
Total number of agents:  128872


### create uniqueTracks (a dictionary strore every unique agents (car, bike, or pedestrain)

In [6]:
# load uniqueTracks data
# uniqueTracks is a dictionary strore each unique object
uniqueTracks = dp.read_uniqueTracks(df_train_small)
uniqueTracks

{101: <data_preprocessing.uniqueTrack at 0x19fe1bf62a0>,
 102: <data_preprocessing.uniqueTrack at 0x19fdfbf54f0>,
 103: <data_preprocessing.uniqueTrack at 0x19fe05be630>,
 104: <data_preprocessing.uniqueTrack at 0x19fe1bf6330>,
 105: <data_preprocessing.uniqueTrack at 0x19fe1bf6f60>,
 106: <data_preprocessing.uniqueTrack at 0x19fe1bf5df0>,
 107: <data_preprocessing.uniqueTrack at 0x19fe1c17560>,
 108: <data_preprocessing.uniqueTrack at 0x19fe1c142c0>,
 109: <data_preprocessing.uniqueTrack at 0x19fe1c16a80>,
 110: <data_preprocessing.uniqueTrack at 0x19fe1c16a50>,
 111: <data_preprocessing.uniqueTrack at 0x19fe1c16960>,
 112: <data_preprocessing.uniqueTrack at 0x19fe1c14080>,
 113: <data_preprocessing.uniqueTrack at 0x19fe1c174d0>,
 114: <data_preprocessing.uniqueTrack at 0x19fe1c14500>,
 115: <data_preprocessing.uniqueTrack at 0x19fe1c14680>,
 201: <data_preprocessing.uniqueTrack at 0x19fe1c16720>,
 202: <data_preprocessing.uniqueTrack at 0x19fe1c14a70>,
 203: <data_preprocessing.uniqu

### information stored in the object

In [7]:
v_max = max(df_train_small['vx']**2 + df_train_small['vy']**2) ** 0.5

In [3]:
#Set Device
device = T.device('cuda:0' if T.cuda.is_available() else 'cpu')

In [5]:
#Initialize the buffer
replay_buffer = Replay_Buffer.ReplayBuffer(37, 2, device, 200000)
save_folder = "buffers/DR_USA_Roundabout_FT"
np.save(f"{save_folder}_state.npy", [])
np.save(f"{save_folder}_action.npy", [])
np.save(f"{save_folder}_next_state.npy", [])
np.save(f"{save_folder}_reward.npy", [])
np.save(f"{save_folder}_not_done.npy", [])
np.save(f"{save_folder}_ptr.npy", [])

In [10]:
%%time
i=0
for unique_id,group in df_train_small.groupby('unique_id'):
    timestamps = group['timestamp_ms'].unique()
    num_timestamps = len(timestamps)
    for timestamp in timestamps[:num_timestamps - 1]:
        frame = group[group['timestamp_ms'] == timestamp]
        ego_id = unique_id
        _, interactive_agents = mdp.get_other_agents_unique_id(ego_id, timestamp, uniqueTracks)
        mdp_tuple = mdp.get_mdp_tuple(ego_id, interactive_agents, timestamp, uniqueTracks, v_max)
        if not not mdp_tuple:
            s, a, r, s_next, the_last_episode = mdp_tuple
            if the_last_episode == False:
                the_last_episode = 0
            else:
                the_last_episode = 1
            replay_buffer.add(s, a, s_next, r, the_last_episode)
            i=i+1
    if i == 200000:
        break

[108, 105, 111]
104 [108, 105, 111]
[108, 105, 111]
104 [108, 105, 111]
[108, 105, 111]
104 [108, 105, 111]


c:\Users\dylan\Desktop\EECS-545\EECS-545-final-project\EECS-545-final-project-ignore_rv_rp\reward.py:168: RuntimeWarning: invalid value encountered in arccos
  alpha = np.arccos((distance_next**2 + distance_closest**2 - distance_next_closest**2)/(2*distance_next*distance_closest))
c:\Users\dylan\Desktop\EECS-545\EECS-545-final-project\EECS-545-final-project-ignore_rv_rp\reward.py:119: RuntimeWarning: invalid value encountered in arccos
  if rotated_rectangles_intersect(((ego_x, ego_y), 0.95 * ego_length, 0.95 * ego_width, np.arccos(ego_psi)),
c:\Users\dylan\Desktop\EECS-545\EECS-545-final-project\EECS-545-final-project-ignore_rv_rp\reward.py:120: RuntimeWarning: invalid value encountered in arccos
  ((interactive_x, interactive_y), 0.95 * interactive_length, 0.95 * interactive_width, np.arccos(interactive_psi))):


[108, 105, 111]
104 [108, 105, 111]
[108, 105, 111]
104 [108, 105, 111]
[108, 105, 111]
104 [108, 105, 111]
[108, 105, 111]
104 [108, 105, 111]
[108, 111, 105]
104 [108, 111, 105]
[108, 111, 105]
104 [108, 111, 105]
[108, 111, 105]
104 [108, 111, 105]
[108, 111, 107]
104 [108, 111, 107]
[108, 111, 107]
104 [108, 111, 107]
[108, 107, 111]
104 [108, 107, 111]
[108, 107, 111]
104 [108, 107, 111]
[108, 107, 111]
104 [108, 107, 111]
[108, 107, 111]
104 [108, 107, 111]
[108, 107, 111]
104 [108, 107, 111]
[108, 107, 111]
104 [108, 107, 111]
[108, 107, 111]
104 [108, 107, 111]
[108, 107, 111]
104 [108, 107, 111]
[108, 107, 111]
104 [108, 107, 111]
[108, 107, 111]
104 [108, 107, 111]
[108, 107, 111]
104 [108, 107, 111]
[108, 107, 111]
104 [108, 107, 111]
[108, 107, 111]
104 [108, 107, 111]
[108, 107, 111]
104 [108, 107, 111]
[108, 107, 111]
104 [108, 107, 111]
[107, 108, 111]
104 [107, 108, 111]
[107, 108, 111]
104 [107, 108, 111]
[107, 108, 111]
104 [107, 108, 111]
[107, 108, 111]
104 [107, 10

c:\Users\dylan\Desktop\EECS-545\EECS-545-final-project\EECS-545-final-project-ignore_rv_rp\reward.py:168: RuntimeWarning: divide by zero encountered in scalar divide
  alpha = np.arccos((distance_next**2 + distance_closest**2 - distance_next_closest**2)/(2*distance_next*distance_closest))


[102, 101, 105]
112 [102, 101, 105]
[102, 101, 105]
112 [102, 101, 105]
[102, 101, 105]
112 [102, 101, 105]
[102, 101, 105]
112 [102, 101, 105]
[102, 101, 105]
112 [102, 101, 105]
[102, 101, 105]
112 [102, 101, 105]
[102, 101, 105]
112 [102, 101, 105]
[102, 101, 105]
112 [102, 101, 105]
[102, 101, 105]
112 [102, 101, 105]
[102, 101, 105]
112 [102, 101, 105]
[102, 101, 105]
112 [102, 101, 105]
[102, 101, 105]
112 [102, 101, 105]
[102, 101, 105]
112 [102, 101, 105]
[102, 101, 105]
112 [102, 101, 105]
[102, 101, 105]
112 [102, 101, 105]
[102, 101, 105]
112 [102, 101, 105]
[102, 101, 105]
112 [102, 101, 105]
[102, 101, 105]
112 [102, 101, 105]
[102, 101, 105]
112 [102, 101, 105]
[102, 101, 105]
112 [102, 101, 105]
[102, 101, 105]
112 [102, 101, 105]
[102, 101, 105]
112 [102, 101, 105]
[102, 101, 105]
112 [102, 101, 105]
[102, 101, 105]
112 [102, 101, 105]
[102, 101, 105]
112 [102, 101, 105]
[102, 101, 105]
112 [102, 101, 105]
[102, 101, 105]
112 [102, 101, 105]
[102, 101, 105]
112 [102, 10

In [ ]:
i

237238

In [ ]:
#Save the Buffer
replay_buffer.save("buffers/DR_USA_Roundabout_FT")

In [ ]:
#Load the Buffer
replay_buffer_load = Replay_Buffer.ReplayBuffer(37, 2, device, 100000)
replay_buffer_load.load(f"buffers/DR_USA_Roundabout_FT")

In [ ]:
replay_buffer_load.sample(1)

(tensor([[ 1.0100e+03,  1.0050e+03,  1.0010e+03,  1.0040e+03,  1.0391e+03,
           9.7264e+02,  0.0000e+00,  0.0000e+00,  2.1610e+00,  5.2000e+00,
           1.8600e+00,  1.0391e+03,  9.7264e+02,  1.0000e+00,  1.0421e+03,
           9.8609e+02, -1.1880e+00,  1.0230e+00,  2.4310e+00,  4.7400e+00,
           1.8000e+00, -1.0000e+00,  1.0247e+03,  9.8049e+02,  6.1450e+00,
           4.1100e-01,  0.0000e+00,  0.0000e+00,  0.0000e+00,  1.0000e+00,
           1.0306e+03,  9.9170e+02,  4.0150e+00,  3.2310e+00,  6.7800e-01,
           4.3900e+00,  1.8900e+00]], device='cuda:0'),
 tensor([[0., 0.]], device='cuda:0'),
 tensor([[ 1.0100e+03,  1.0050e+03,  1.0010e+03,  1.0040e+03,  1.0391e+03,
           9.7264e+02,  0.0000e+00,  0.0000e+00,  2.1610e+00,  5.2000e+00,
           1.8600e+00,  1.0391e+03,  9.7264e+02,  1.0000e+00,  1.0420e+03,
           9.8620e+02, -1.1600e+00,  1.0080e+00,  2.4260e+00,  4.7400e+00,
           1.8000e+00, -1.0000e+00,  1.0253e+03,  9.8053e+02,  6.1380e+00,
      